In [1]:
import pandas as pd

df =  pd.read_csv(r'C:\Users\krmch\OneDrive\Desktop\10manhattenyellowtaxis.csv')

In [2]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [3]:
print(len(df))

74658534


In [4]:
df['tpep_pickup_datetime']  = pd.to_datetime(df['tpep_pickup_datetime'] ,  format ='%m/%d/%Y %I:%M:%S %p')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'] ,  format ='%m/%d/%Y %I:%M:%S %p')

In [5]:
print(df.dtypes)

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object


In [6]:
print(df.isnull().sum())

VendorID                   88879
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count            88879
trip_distance                  0
RatecodeID                 88879
store_and_fwd_flag         88879
PULocationID                   0
DOLocationID                   0
payment_type               88879
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     4306303
dtype: int64


In [7]:
df = df.dropna(subset=['passenger_count'])

In [8]:
print(len(df))

74569655


In [9]:
print(df.isnull().sum())

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count                0
trip_distance                  0
RatecodeID                     0
store_and_fwd_flag             0
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     4306303
dtype: int64


congestion_surcharge has a mode  of 2.50 based on the histogram in the data quality report. As such I will impute the mode for the missing values in congestion_surcharge.

In [10]:
print(df['congestion_surcharge'].unique())

[ 2.5   0.   -2.5   0.75  2.75  2.25 -0.75  0.5   1.     nan -1.5   1.5
  3.    4.5 ]


In [11]:
print((df['congestion_surcharge'] ==  2.50).sum())

65153407


In [12]:
df['congestion_surcharge'].fillna(2.50, inplace=True)

In [13]:
print(df.isnull().sum())

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
dtype: int64


dealing with negative values 

In [14]:
x = [
    "payment_type", "VendorID", "passenger_count", "trip_distance",
    "RatecodeID","PULocationID","DOLocationID","fare_amount",
    "extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge",
    "total_amount", "congestion_surcharge"
]


d = {}


for feature in x:
    if feature in df.columns:
        d[feature] = (df[feature] < 0).sum()
        
for feature,  count in d.items():
    print(f'{feature}: {count}')

payment_type: 0
VendorID: 0
passenger_count: 0
trip_distance: 23
RatecodeID: 0
PULocationID: 0
DOLocationID: 0
fare_amount: 131705
extra: 65222
mta_tax: 129750
tip_amount: 699
tolls_amount: 1860
improvement_surcharge: 131677
total_amount: 131696
congestion_surcharge: 114075


In [15]:
for i in x:
    if i in df.columns:
        df[i] = df[i].apply(lambda y: abs(y) if y < 0 else y)

In [16]:
x = [
    "payment_type", "VendorID", "passenger_count", "trip_distance",
    "RatecodeID","PULocationID","DOLocationID","fare_amount",
    "extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge",
    "total_amount", "congestion_surcharge"
]


d = {}


for feature in x:
    if feature in df.columns:
        d[feature] = (df[feature] < 0).sum()
        
for feature,  count in d.items():
    print(f'{feature}: {count}')

payment_type: 0
VendorID: 0
passenger_count: 0
trip_distance: 0
RatecodeID: 0
PULocationID: 0
DOLocationID: 0
fare_amount: 0
extra: 0
mta_tax: 0
tip_amount: 0
tolls_amount: 0
improvement_surcharge: 0
total_amount: 0
congestion_surcharge: 0


Next we are going to remove values that are incoherent based on domain knowledge. As discussed in the data quality report NYC taxis can only take 6 passengers including a child under 7 and the base fare for 2019 was 2.50$

we will count hte number of rows with incoherent values. If many compared to the large 80 million row dataset we may choose imputation or if there are only a few rows we may just remove them.

References

Bloomberg (2022) 'NYC Taxi Cab Fares to Rise 23% in First Increase Since 2012', Bloomberg. Available at: https://www.bloomberg.com/news/articles/2022-11-15/nyc-taxi-cab-fares-to-rise-23-in-firs t-increase-since-2012 (Accessed: 7 July 2023).
NYC.gov (n.d.) 'TLC Rules', NYC.gov. Available at: https://www.nyc.gov/site/tlc/about/tlc-rules.page (Accessed: 7 July 2023).

In [17]:
o1 = df[df['passenger_count'] > 6 ].shape[0]
print(o1)

296


In [18]:
df = df[ df['passenger_count']  <= 6]

In [19]:
o1 = df[df['passenger_count'] > 6 ].shape[0]
print(o1)

0


In [20]:
o2 = df[df['passenger_count'] == 0].shape[0]
print(o2)

1346863


In [21]:
df['passenger_count'].replace(0 , 1 ,  inplace=True)

In [22]:
o2 = df[df['passenger_count'] == 0].shape[0]
print(o2)

0


In [23]:
o3 = (df['tpep_pickup_datetime']  >=  df['tpep_dropoff_datetime']).sum()
print(o3)

2910


In [24]:
df =   df[df['tpep_pickup_datetime']   < df['tpep_dropoff_datetime']]

In [25]:
start = pd.Timestamp('2018-12-31')
end = pd.Timestamp('2020-01-01')

In [26]:
out = ((df['tpep_pickup_datetime'] < start)  | (df['tpep_pickup_datetime'] > end)) | ((df['tpep_dropoff_datetime'] < start) |  (df['tpep_dropoff_datetime']  > end))
print(out.sum())

1733


In [27]:
df = df[((df['tpep_pickup_datetime']  >= start)    & (df['tpep_pickup_datetime']  <= end))   & ((df['tpep_dropoff_datetime'] >= start)    & (df['tpep_dropoff_datetime']  <= end))]

In [28]:
out = ((df['tpep_pickup_datetime'] < start)  | (df['tpep_pickup_datetime'] > end)) | ((df['tpep_dropoff_datetime'] < start) |  (df['tpep_dropoff_datetime']  > end))
print(out.sum())

0


In [29]:
print((df['VendorID'] == 4).sum())

241732


even though theres no mention of a 4 for vendorID in the data dictionary I find it highly unlikely that this particular value was entered 250,000 times incorrectly and no other incorrect values were added. As such I will keep the vendorIDs that are 4 because I suspect it constitutes a valid 3rd vendor.

In [30]:
print((df['RatecodeID'] == 99).sum())

2121


In [31]:
df['RatecodeID'].replace(99 , 1 , inplace=True)

In [32]:
print((df['RatecodeID'] == 99).sum())

0


In [33]:
print(len(df))

74564716


In [34]:
print(df['improvement_surcharge'].unique())

[0.3  0.   0.03]


0.03 is likely a mistake since data dictionary states its a standard 30% tax.

In [35]:
print((df['improvement_surcharge'] == 0.03).sum())

5


In [36]:
df['improvement_surcharge'].replace(0.03 , 0.3 , inplace=True)

In [37]:
print((df['improvement_surcharge'] == 0.03).sum())

0


In [38]:
print(df['congestion_surcharge'].unique())
print(df['mta_tax'].unique())

[2.5  0.   0.75 2.75 2.25 0.5  1.   1.5  3.   4.5 ]
[5.000e-01 0.000e+00 3.500e-01 6.080e+01 2.745e+01 3.300e+00 9.300e-01
 3.300e-01 1.100e+00 3.080e+01 8.600e-01 1.440e+00 1.539e+01 6.800e+00
 2.452e+01 2.240e+00 2.780e+00 5.000e-02 3.800e+00 1.942e+01 2.500e-01
 2.750e+00 8.500e-01 8.700e-01 3.200e-01 3.000e+00 3.253e+01 8.900e-01
 2.439e+01]


In [39]:
# count number of rows for congestion surcharge being 0 or 2.5

print((df['congestion_surcharge'] ==  2.50).sum())
print((df['congestion_surcharge'] ==  0).sum())

69569613
4992948


In [40]:
mode_congestion_surcharge = df['congestion_surcharge'].mode()[0]

# since almost all the values are in one of these categories congestion surchage is likely a standard charge
df.loc[~df['congestion_surcharge'].isin([0, 2.50]), 'congestion_surcharge'] = mode_congestion_surcharge

In [41]:
mode_mta_tax = df['mta_tax'].mode()[0]

# mta tax is a standard 50 cent charge 
df.loc[~df['mta_tax'].isin([0, 0.50]), 'mta_tax'] = mode_mta_tax

In [42]:

def detectouts(z):
    Q1 = z.quantile(0.25)
    Q3 = z.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((z < (Q1 - 1.5 * IQR)) | (z > (Q3 + 1.5 * IQR)))
    return outliers.sum()

outcols1 = ['trip_distance', 'fare_amount', 'tip_amount',  'tolls_amount', 'total_amount']


outcounts = {col: detectouts(df[col]) for col in outcols1}

outcounts


{'trip_distance': 6466530,
 'fare_amount': 5076973,
 'tip_amount': 2247761,
 'tolls_amount': 2653029,
 'total_amount': 5100167}

In [44]:
def cappy(df, w):
    
    
    Q1 = df[w].quantile(0.25)
    Q3 = df[w].quantile(0.75)
    
    #subtract the 2 
    IQR = Q3 - Q1
    
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    df[w] = df[w].clip(lower=lower_bound, upper=upper_bound)

    return df

outcols = ['trip_distance',   'fare_amount', 'tip_amount',   'tolls_amount', 'total_amount']

for col in outcols:
    
    
    df =  cappy(df, col)

df.head()




,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2019-03-30 02:55:16,2019-03-30 03:07:53,1.0,1.57,1.0,N,107,90,1.0,6.5,0.5,0.5,1.70,0.0,0.3,12.00,2.5
1,2.0,2019-03-30 03:23:54,2019-03-30 03:28:05,1.0,1.10,1.0,N,249,234,1.0,5.5,0.5,0.5,1.50,0.0,0.3,10.80,2.5
2,1.0,2019-03-30 03:44:03,2019-03-30 03:52:47,1.0,2.00,1.0,N,137,246,1.0,8.5,3.0,0.5,1.00,0.0,0.3,13.30,2.5
3,2.0,2019-03-30 03:03:05,2019-03-30 03:11:01,1.0,1.76,1.0,N,263,229,1.0,8.5,0.5,0.5,2.46,0.0,0.3,14.76,2.5
4,2.0,2019-03-30 03:04:36,2019-03-30 03:20:21,1.0,1.89,1.0,N,158,144,1.0,11.5,0.5,0.5,3.06,0.0,0.3,18.36,2.5


In [45]:
print(df.duplicated().sum())

13242


In [46]:
print(len(df))

74564716


In [47]:
df.drop_duplicates(inplace=True)

In [48]:
print(len(df))

74551474


In [49]:
df.to_csv(r'C:\Users\krmch\OneDrive\Desktop\25manhattenyellowtaxis.csv' ,  index=False)